# Function Calling in OpenAI

In [1]:
from openai import OpenAI
from dotenv import load_dotenv,find_dotenv
import json
_ : bool = load_dotenv(find_dotenv())
client : OpenAI = OpenAI()

In [2]:
def get_current_weather(location: str, unit:str='fahrenheit')->str:
    """get the current weather in given loaction"""
    if "tokyo" in location.lower():
        return json.dumps({"loaction": "tokyo", "temperature":"10", "unit":"celcius"})
    elif "new york" in location.lower():
        return json.dumps({"location": "new york", "temperature":"16","unit": "fahrenheit"})
    elif "san francisco" in location.lower():
        return json.dumps({"location":"san francisco", "temperature": "20", "unit":"fahrenheit"})
    else:
        return json.dumps({"location": location,"temperature": "unknown"})



In [4]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
# from openai.types.chat.chat_completion import ChatCompletionMessageParam, ChatCompletionMessageParam

def run_conversation(main_request: str)->str:
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": main_request}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))

    tools_calls = response_message.tool_calls
    display(list(tools_calls))
    # Step 2 check if model want to call a function
    if tools_calls:
        # step 3 call the function
        available_functions = {
       "get_current_weather": get_current_weather
     }
        messages.append(response_message)

        for tool_call in tools_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_reponse = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )

            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": tool_call.function.name,
                    "content": function_reponse,
                }
            ),
    display("second message", list(messages))
    second_response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
    )
    display("print seond reponse", dict(second_response))
    response_msg: ChatCompletionMessage =  second_response.choices[0].message.content
    return response_msg
            

In [5]:
run_conversation("What's the weather like in San Francisco?")

'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_mFIjSPsCskjjmpZsiETZZDzm', function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_80Q1PDF6dfMzaornGENQRjGA', function=Function(arguments='{"location": "San Francisco, CA", "unit": "fahrenheit"}', name='get_current_weather'), type='function')]}

[ChatCompletionMessageToolCall(id='call_mFIjSPsCskjjmpZsiETZZDzm', function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_80Q1PDF6dfMzaornGENQRjGA', function=Function(arguments='{"location": "San Francisco, CA", "unit": "fahrenheit"}', name='get_current_weather'), type='function')]

'second message'

[{'role': 'user', 'content': "What's the weather like in San Francisco?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_mFIjSPsCskjjmpZsiETZZDzm', function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_80Q1PDF6dfMzaornGENQRjGA', function=Function(arguments='{"location": "San Francisco, CA", "unit": "fahrenheit"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_mFIjSPsCskjjmpZsiETZZDzm',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "san francisco", "temperature": "20", "unit": "fahrenheit"}'},
 {'tool_call_id': 'call_80Q1PDF6dfMzaornGENQRjGA',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "san francisco", "temperature": "20", "unit": "fahrenheit"}'}]

'print seond reponse'

{'id': 'chatcmpl-8Q1ZhjLzxjhFnCBr94rgFojLuW1Mn',
 'choices': [Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The current weather in San Francisco is 20°C (68°F).', role='assistant', function_call=None, tool_calls=None))],
 'created': 1701213273,
 'model': 'gpt-3.5-turbo-1106',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_eeff13170a',
 'usage': CompletionUsage(completion_tokens=14, prompt_tokens=131, total_tokens=145)}

'The current weather in San Francisco is 20°C (68°F).'